# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', 70)
import os
import sys
import re

from sqlalchemy import create_engine
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_messages_database.db')
df = pd.read_sql_table('disaster_messages_table', engine)


In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
#Y = df.drop(['id','message','original','genre'], axis=1)
Y = df.iloc[:,4:]

In [6]:
X.head(n=3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

In [7]:
Y.head(n=3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """
    function to extract tokens from the input text
    
    Input:
        text: Text to be tokenized
    Output:
        clean_tokens: List of tokens extracted
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [10]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    class for extracting the starting verb 
    
    This class creating a new feature for the ML classifier by extracting the starting verb
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
    
pipeline_adaboost = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [12]:
def evaluate_metrics(array_gt, array_pred, column_names):
    """function for evaluating different metrics of the model performance
    
    inputs:
    array_gt: array containing the ground truth labels.
    array_pred: array containing predicted labels.
    column_names: list of strings. list containing names for each of the predicted array fields.
       
    Returns:
    data_metrics: accuracy, precision, recall and f1 scores.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(column_names)):
        accuracy = accuracy_score(array_gt[:, i], array_pred[:, i])
        precision = precision_score(array_gt[:, i], array_pred[:, i], average = 'weighted')
        recall = recall_score(array_gt[:, i], array_pred[:, i],average = 'weighted')
        f1 = f1_score(array_gt[:, i], array_pred[:, i], average = 'weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = column_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics  

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model = pipeline.fit(X_train, y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
#y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [15]:
 print(evaluate_metrics(np.array(y_test), y_pred_test, list(Y.columns.values)))

                        Accuracy  Precision    Recall        F1
related                 0.793714   0.773632  0.793714  0.773289
request                 0.879005   0.875883  0.879005  0.859341
offer                   0.995117   0.990259  0.995117  0.992682
aid_related             0.728868   0.734159  0.728868  0.716958
medical_help            0.920201   0.897673  0.920201  0.890870
medical_products        0.950717   0.939248  0.950717  0.929986
search_and_rescue       0.971620   0.965587  0.971620  0.959789
security                0.983369   0.971837  0.983369  0.975719
military                0.968416   0.952265  0.968416  0.956866
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.944614   0.940581  0.944614  0.927847
food                    0.919896   0.913918  0.919896  0.905826
shelter                 0.928593   0.920519  0.928593  0.911732
clothing                0.984895   0.980278  0.984895  0.978244
money                   0.977266   0.977

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:
# other alternatives for calculating metrics 
#print(classification_report(np.hstack(y_test),np.hstack(y_pred_test)))
#print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

In [ ]:
#print(classification_report(np.array(y_train), y_pred_train, target_names=list(Y.columns.values)))

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb_transformer', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'feat

In [17]:
parameters = {
    'clf__estimator__n_estimators':[10, 20],
    'clf__estimator__min_samples_split':[4, 7]
}

In [18]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [19]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [4, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
y_predictions_test = cv.predict(X_test)
print(evaluate_metrics(np.array(y_test), y_predictions_test, list(Y.columns.values)))

                        Accuracy  Precision    Recall        F1
related                 0.802258   0.787711  0.802258  0.767893
request                 0.892890   0.891399  0.892890  0.878607
offer                   0.995117   0.990259  0.995117  0.992682
aid_related             0.755264   0.754563  0.755264  0.750917
medical_help            0.921269   0.901548  0.921269  0.893105
medical_products        0.952853   0.946648  0.952853  0.934655
search_and_rescue       0.970857   0.961502  0.970857  0.958057
security                0.983522   0.967612  0.983522  0.975502
military                0.970857   0.963521  0.970857  0.958795
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.949344   0.947857  0.949344  0.935812
food                    0.927220   0.922743  0.927220  0.916458
shelter                 0.934696   0.930526  0.934696  0.920462
clothing                0.985810   0.983736  0.985810  0.980271
money                   0.976655   0.969

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
pipeline_adaboost.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_p

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
# trying another model: AdaBoost and experimenting with parameters

parameters_new = {'clf__estimator__learning_rate': [0.3, 0.5],
                  'clf__estimator__n_estimators': [10, 20]}

cv_new = GridSearchCV(pipeline_adaboost, param_grid = parameters_new)

# Find best parameters
np.random.seed(42)
model_new = cv_new.fit(X_train, y_train)

In [28]:
model_new.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__learning_rate': [0.3, 0.5], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
y_pred_test_new = model_new.predict(X_test)
print(evaluate_metrics(np.array(y_test), y_pred_test_new, list(Y.columns.values)))

                        Accuracy  Precision    Recall        F1
related                 0.761977   0.657978  0.761977  0.659462
request                 0.889075   0.885204  0.889075  0.874905
offer                   0.995117   0.990259  0.995117  0.992682
aid_related             0.722002   0.729222  0.722002  0.707762
medical_help            0.922490   0.908751  0.922490  0.893444
medical_products        0.955142   0.947599  0.955142  0.941238
search_and_rescue       0.972231   0.966445  0.972231  0.961586
security                0.983827   0.978792  0.983827  0.976246
military                0.972231   0.966937  0.972231  0.962181
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.957583   0.954092  0.957583  0.955118
food                    0.947055   0.945798  0.947055  0.946329
shelter                 0.944919   0.940537  0.944919  0.937769
clothing                0.987031   0.984633  0.987031  0.983552
money                   0.977266   0.970

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [30]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(model_new, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.